In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
from scipy.interpolate import BSpline
from selectinf.Simulation.spline import b_spline
from selectinf.Simulation.spline_instance import (generate_gaussian_instance_nonlinear, 
                                                  generate_gaussian_instance_from_bspline,
                                                  generate_gaussian_instance_from_bspline_interaction)
from selectinf.group_lasso_query import (group_lasso,
                                         split_group_lasso)
import regreg.api as rr
from selectinf.base import selected_targets
from selectinf.base import restricted_estimator
import scipy.stats

from selectinf.Simulation.test_group_lasso_simulation import (calculate_F1_score,
                                                              naive_inference,
                                                              randomization_inference,
                                                              data_splitting)

/Users/yilingh/Desktop/PhD/SI_Codes/SI-Interaction/env3/lib/python3.9/site-packages/regreg/atoms/slope.py:16: UserWarning: unable to import isotonic regression from sklearn, using a pure python implementation
  warn('unable to import isotonic regression from sklearn, using a pure python implementation')


In [11]:
(design, data_interaction, Y, Y_mean, data_combined,
 groups, active, active_inter_adj, active_inter_list) \
    = generate_gaussian_instance_from_bspline_interaction(n=500, p_nl=50, p_l=0,
                                                          s_nl=2, s_l=0,
                                                          nknots=8, degree=2, SNR=1,
                                                          center=False, scale=True, 
                                                          random_signs=True,
                                                          intercept=False, structure='allpairs', s_interaction=5)

Equally spaced quantile knots used.
19 33
16 23
2 16
35 49
29 49
noise_sd: 0.19422509549244946


In [12]:
def sample_interaction_pairs_hierarchy(p, s_inter, active_idx, mode='weakhierarchy'):
    active_idx -= 1
    
    # Construct collection of candidate pairs
    pairs = []
    if mode == 'weakhierarchy':
        for i in range(p):
            for j in range(i + 1, p):
                if i in active_idx or j in active_idx:
                    pairs.append((i, j))
    elif mode == 'stronghierarchy':
        for i in range(p):
            for j in range(i + 1, p):
                if i in active_idx and j in active_idx:
                    pairs.append((i, j))
    pairs = np.array(pairs)

    # Sample some interaction pairs
    pairs_smpl_idx = np.random.choice(np.arange(len(pairs)), s_inter, replace=False)
    pairs_smpl = pairs[pairs_smpl_idx]

    active_inter = np.zeros((p,p))
    for i in range(pairs_smpl.shape[0]):
        pair = pairs_smpl[i]
        active_inter[pair[0], pair[1]] = 1
        print(pair[0], pair[1])

    return active_inter, pairs_smpl

In [13]:
active_inter, pairs_smpl\
    = sample_interaction_pairs_hierarchy(10, 5, np.array([1,5,8,10]), mode='stronghierarchy')

7 9
4 7
0 9
4 9
0 7
